<a href="https://colab.research.google.com/github/ponponxx/Keyence_LJXSample_OpenCV/blob/main/3D_Mesh_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2
import open3d as o3d
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
#input_path="C:\\Users\\KW0158W10\\Desktop\\"
#output_path="C:\\Users\\KW0158W10\\Desktop\\"
#dataname="sample_w_normals.xyz"
#point_cloud= np.loadtxt(input_path+dataname,skiprows=1)

In [ ]:
Data = cv2.imread('C:\\Users\\KW0158W10\\Desktop\\1234_height.tif',-1)
rows, cols = Data.shape[0:2]
newrows = int(rows/8)
newcols = int(cols/8)
DataS = cv2.resize(Data , (newcols, newrows))
Datanumbers = newrows*newcols
print('newrows / Y = ',newrows)
print('newcols / X = ',newcols)

newrows / Y =  750
newcols / X =  400


In [ ]:
PCDdata = np.zeros((Datanumbers+1 ,6),dtype=np.uint16)
imgH = np.zeros((newrows,newcols),dtype=np.uint8)
imgH[:,:] = 255
imgS = np.zeros((newrows,newcols),dtype=np.uint8)
imgS[:,:] = 255
imgG = np.zeros((newrows,newcols),dtype=np.uint8)
imgG[:,:] = 255

In [ ]:
RED =  33000
BLUE = 31500
R = (RED-BLUE)/140
for X in range(newrows):
    for Y in range(newcols):
        newV = int((DataS[X,Y]-BLUE)/R)
        if newV > 140 :
            imgH[X,Y] = 0
        elif newV < 0:
            imgH[X,Y] = 140
        else:
            imgH[X,Y] = 140-newV

FullPicHSV = cv2.merge([imgH,imgS,imgG])
FullPicBGR = cv2.cvtColor(FullPicHSV,cv2.COLOR_HSV2BGR)
FullPicBGR.shape

(750, 400, 3)

In [ ]:
#winname = "CheckColor"
#cv2.namedWindow(winname)        # Create a named window
#cv2.moveWindow(winname, 40,30)  # Move it to (40,30)
#cv2.imshow(winname, FullPicBGR)
#cv2.waitKey()
#cv2.destroyAllWindows()

In [ ]:
i = 0
for X in range(newcols):
    for Y in range(newrows):
        PCDdata[i] = (X,Y,DataS[Y,X]/256,FullPicBGR[Y,X,2],FullPicBGR[Y,X,1],FullPicBGR[Y,X,0])
        i+=1

In [ ]:
pcd = o3d.geometry.PointCloud()
#pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
pcd.points = o3d.utility.Vector3dVector(PCDdata[:,:3])
pcd.colors = o3d.utility.Vector3dVector(PCDdata[:,3:6])
#pcd.normals = o3d.utility.Vector3dVector(point_cloud[:,6:9])
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))


In [ ]:
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 控制代碼無效。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 控制代碼無效。 


In [ ]:
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

In [ ]:
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))

In [ ]:
dec_mesh = bpa_mesh.simplify_quadric_decimation(100000)

In [ ]:
output_path="C:\\Users\\KW0158W10\\Desktop\\"
o3d.io.write_triangle_mesh(output_path+"bpa_mesh.stl", dec_mesh)

True

In [ ]:
網格處理用Command
#dec_mesh.remove_degenerate_triangles()
#dec_mesh.remove_duplicated_triangles()
#dec_mesh.remove_duplicated_vertices()
#dec_mesh.remove_non_manifold_edges()

In [ ]:
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8, width=0, scale=1.1, linear_fit=False)[0]

In [ ]:
bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)
p_mesh_crop = o3d.geometry.TriangleMesh.compute_triangle_normals (p_mesh_crop)

In [ ]:
output_path="C:\\Users\\KW0158W10\\Desktop\\"
o3d.io.write_triangle_mesh(output_path+"1234_height.stl", p_mesh_crop)

True